This is a script written by Zach Monge to create representational similarity matrices (RSMs) from layers of the AlexNet (pretrained on the ImageNet dataset).


I ran this script on Google Cloud Platform with a NVIDIA Tesla K80 GPU. Parts of this script were taken from https://github.com/CSAILVision/places365

In [1]:
# Importing functions. For the deep learning library I am using PyTorch
import torch
from torch.autograd import Variable as V
import torchvision.models as models
from torchvision import transforms as trn
from torch.nn import functional as F
import os
from PIL import Image
import torch.nn as nn
import numpy as np
import pandas as pd
from scipy.io import savemat
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only difference


In [2]:
# Importing AlexNet model pretrained on ImageNet
alexnet = models.alexnet(pretrained=True)
model = alexnet

# Shutting off dropoff since we are only evaluating and not training the model
model.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d (3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), dilation=(1, 1))
    (3): Conv2d (64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), dilation=(1, 1))
    (6): Conv2d (192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d (384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), dilation=(1, 1))
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096)
    (2): ReLU(inplace)
    (3): Dropout(p=0.5)
    (4): Linear(in_features=4096, out_features=4096)
    (5): ReLU(inplace)
    (6): Linea

In [3]:
# Specifying the Path in which the images were saved
PATH = '/home/zachm/neuro/resized/'

# Importing the CSV file that contains the id number for each image and the image filename
image_df=pd.read_csv(f'{PATH}image_ids.csv')
image_df.head()

,id,image
0,1,2751.jpg
1,2,8480.jpg
2,3,1275.jpg
3,4,6243.jpg
4,5,9050.jpg


In [4]:
# load the image transformer
centre_crop = trn.Compose([
        trn.Resize((256,256)),
        trn.CenterCrop(224),
        trn.ToTensor(),
        trn.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

Now will submit each image to AlexNet and loop through each layer to obtain the activation values. For each image at each layer, I output the activation values, which are vectorized and then stored in a Pandas dataframe. So the output of the below cell is for each layer a Pandas dataframe, where each column is the activation values for an image.

First this is done for the covolutional layers (first five layers) and then the fully-connected layers (last three layers).

In [4]:
# First for the convolutional layers
for layer_number in range(len(model.features)):
    layer_number_count = layer_number+1
    
    # Create a temporary model in which the last layer is the layer of interest within the loop
    temp_model=nn.Sequential(*list(model.features.children())[0:layer_number_count])
    
    # Creating df for layer
    vars()[f'layer_{layer_number_count}']=pd.DataFrame()
    
    # Forward passing the image through the model
    for img_name in image_df['image']:
        
        #Opening image
        try:
            img = Image.open(f'{PATH}{img_name}')
        except:
            img = Image.open(f'{PATH}{img_name[:-4]}.JPG')
            
        input_img = V(centre_crop(img).unsqueeze(0), volatile=True)

        # Forward pass image through the model
        feature_values = temp_model.forward(input_img)
        
        #Vectorizing torch tensor
        element_size=np.prod(np.array(feature_values.data.size()))
        feature_values_vec=feature_values.view(np.int(element_size),1)
        feature_values_vec_np=feature_values_vec.data.numpy()
        
        # Putting activation values into the layer df
        vars()[f'layer_{layer_number_count}'][img_name]=feature_values_vec_np.flatten()

#Now for the fully connected layers   
for layer_number in range(len(model.classifier)):
    
    # Resetting the model here. I found if I did not do this the script did not run properly
    alexnet = models.alexnet(pretrained=True)
    model = alexnet
    model.eval()
    
    # Just a few small specifications such as the layer number
    layer_number_count = layer_number_count+1
    layer_number_count_iteration = layer_number+1
    temp_model_classifier_layers = model
    
    # Create a temporary model in which the last layer is the layer of interest within the loop
    new_classifier=nn.Sequential(*list(temp_model_classifier_layers.classifier.children())[0:layer_number_count_iteration])
    temp_model_classifier_layers.classifier = new_classifier
    
    # Creating df for layer
    vars()[f'layer_{layer_number_count}']=pd.DataFrame()
    
    # Forward passing the image through the model
    for img_name in image_df['image']:
        
        #Opening image
        try:
            img = Image.open(f'{PATH}{img_name}')
        except:
            img = Image.open(f'{PATH}{img_name[:-4]}.JPG')
        input_img = V(centre_crop(img).unsqueeze(0), volatile=True)

        # Forward pass image through the model
        feature_values = temp_model_classifier_layers.forward(input_img)
        
        #Vectorizing torch tensor
        element_size=np.prod(np.array(feature_values.data.size()))
        feature_values_vec=feature_values.view(np.int(element_size),1)
        feature_values_vec_np=feature_values_vec.data.numpy()
        
        # Putting activation values into the layer df
        vars()[f'layer_{layer_number_count}'][img_name]=feature_values_vec_np.flatten()
        
# Taking the last layer and running it through a softmax activation function. This is commonly done within deep learning.
last_layer_number = layer_number_count
layer_number_count = layer_number_count+1
# Creating df for the last layer
vars()[f'layer_{layer_number_count}']=pd.DataFrame()
for img_name in image_df['image']:
    vars()[f'layer_{layer_number_count}'][img_name]=softmax(vars()[f'layer_{last_layer_number}'][img_name].values)

For each layer, will now take the activation values for each image within each dataframe, and correlate them with each other. This will create a stimuli model for each layer.

Here I have it set to do a Pearson correlation and to place NaNs on the diagonal, but this can be easily changed by making minor edits to the code below.

In [5]:
for i in range(layer_number_count):
    i = i+1
    vars()[f'layer_{i}_corr_matrix'] = vars()[f'layer_{i}'].corr(method = 'pearson')
    
    # Places the value 5 on the diagonol so will be NaNed
    np.fill_diagonal(vars()[f'layer_{str(i)}_corr_matrix'].values,5)
    vars()[f'layer_{i}_corr_matrix_nan'] = vars()[f'layer_{i}_corr_matrix'][vars()[f'layer_{i}_corr_matrix']<5]

To run the Representatioanl Similarity Analysis, I typically using scripts written within Matlab (https://github.com/brg015). So I output the correlation matrices into a .mat file

In [6]:
# Exporting numpy matrixes to MATLAB .mat files

# Order of image ID in matrices. This wll be specific to your images. 
# Within the current study, the image IDs ranged from 1 to 420.
stim_ID_num=np.arange(1,421)

# Specify path where to save .mat files
matlab_data_path = '/home/zachm/neuro/RDMs/EmoLOP_AlexNet/'

# Exporting each layer to .mat file. The two variables contained within the MAT file are R (the correlation matrix)
# and stim_ID_num (the order of iamge ID in matrices). Again, this was done to work with the MATLAB scripts
for j in range(last_layer_number+1):
    layer_number_count = j+1
    savemat(f'{matlab_data_path}layer_{layer_number_count}_corr_matrix_nan',{"R": vars()['layer_'+str(layer_number_count)+'_corr_matrix_nan'].values,"stim_ID_num":stim_ID_num})